<a href="https://colab.research.google.com/github/elliemci/vision-transformer-models/blob/main/model_deployment/mri_classification_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying a MRI classification app with Huggingface and Gradio

In [ ]:
!pip install --upgrade gradio

In [ ]:
!pip install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ColabNotebooks/ExplainableAI/model_deployment

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/ExplainableAI/model_deployment


## 1. Create a Gradio App that exposes the MRI brain tumor classification model

In [ ]:
import gradio as gr

def update_gallery(images):
    """Update the gallery with the uploaded images"""
    return images


with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center;'>MRI Brain Tumor Classification App</h1>")
    with gr.Column():
        image_input = gr.Files(label="Upload MRI Images",
                              file_count="multiple",
                              type="filepath")

        gallery = gr.Gallery(label="MRI Brain Images")

    # update the gallery images when image_input changes
    image_input.change(
        fn=update_gallery,
        inputs=[image_input],
        outputs=[gallery])


demo.launch(debug=True)

## 2. Gradio UI for MRI classification

### Load a vision transforer model trained for MRI classification and image processor

### Immage class prediction function

### Gradio UI for uploading image, making and displaying the prediction

## 3. Gradio UI for MEI segmentation

### Load image segmentation model and image processor

### Image segmentation prediction function

### Gradio UI for uploading image and making a prediction

### Gradio components to display segmentation result

## 4. Gradio Interface

### App contents

#### Text section

#### Gallery section

#### Image analysis section

### App